In [1]:
# Plans for the future:
#      -Add in a grid pattern
#      -Add in a circular pattern + noise to see if it forms a ring around a planetary body
#      -Add in stationary masses
#      -Record times
# Add in these features, then start looking at tree methods. Once you can implement that, then compare with original

In this notebook we are just going to be looking at a small and simple gravity simulation with three point masses. This is just an experiment into physical simulations so its nothing fancy. Infact it uses the worst updating algorithm, of just following each particle and updating its positions/interactions on every iteration. Very computationally heavy, but that isnt the purpose of this notebook. I intend to look into optimized designs in the future but this is more of a proof of concept notebook.

In [2]:
# Run this line below into the console while in the directory of where the images will be saved.

# ffmpeg -start_number 0 -framerate 60 -i graph%01d.png video.webm

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import cmath
import decimal
import os
import sys
import time

In [4]:
# We have this cell here to call the dataAnalysisLibrary module from one place rather than copy pasting into many other places
# The path here is my own path, will need to change it for whoever is using
# Want to find a way to do this automatically so the reader doesnt have to worry about it
# I really need to generalize this...

sys.path.insert(0,'./simulationLibrary')
import simulationLibrary as sim

Now lets try to rewrite everything so that we can generalize and produce randomly sized and spaced bodies. That would be the goal, to just run 'gravitySim()' and let it go, you know?

In [3]:
# The gravitational body in question
# creates an instance of a gravitational point mass.
# position, velocity and acceleration should all be 1x2 arrays
class PointMassBody:
    def __init__(self,mass,position,velocity,acceleration):
        self.mass = mass
        self.position = position
        self.velocity = velocity
        self.acceleration = acceleration
        self.path = np.array([position])

In [4]:
def randParticleGravity():
    #Declare randomly assigned mass
    mass = np.random.normal(loc=1*10**12,scale=100000000000)
    
    #Declare randomly assigned positions and make sure it doesnt go out of bounds
    posx = np.random.normal(scale=5)
    posy = np.random.normal(scale=5)
    if posx >= 30:
        posx = 30
    if posy >= 30:
        posy = 30
    position = np.array([posx,posy])
    
    #Declare randomly assigned positions and make sure it doesnt go out of bounds
    velx = np.random.normal(scale=0.2)
    vely = np.random.normal(scale=0.2)
    velocity = np.array([velx,vely])

    #Generate and return the particle
    particle = PointMassBody(mass,position,velocity,np.array([0,0]))
    return(particle)

In [5]:
def generateParticles(num):
    pointMassArray=np.zeros(num,dtype='object')
    for i in range(len(pointMassArray)):
        pointMassArray[i]=randParticleGravity()
    return(pointMassArray)

In [6]:
def calculateGravity(obj1,obj2):
    G = 6.674*pow(10,-11)
    F12Hat = (obj2.position-obj1.position)/np.linalg.norm(obj2.position-obj1.position)
    F12 = ((G*obj1.mass*obj2.mass)/pow(np.linalg.norm(obj2.position-obj1.position),2))*F12Hat
    return(F12)

In [7]:
def update(objects,dt=1/30):

    # Initialize a 'force matrix', populate it with the forces of the corresponding index. So, at poition (i,j) is F_ij
    forceMatrix=np.zeros((len(bodies),len(bodies),2))
    for j in range(len(bodies)):
        for i in range(len(bodies)):
            if i == j:
                forceMatrix[i][j]=0
            else:
                forceMatrix[i][j]=calculateGravity(bodies[i],bodies[j])
    
    # Calculate Total Forces
    totForceArray = np.zeros((len(objects),2))
    acceleration = np.zeros((len(objects),2))
    for i in range(len(objects)):
        totForceArray[i]=forceMatrix[i].sum(axis=0)
        acceleration[i]=(1/objects[i].mass)*totForceArray[i]
        objects[i].acceleration = acceleration[i]
        objects[i].velocity = objects[i].velocity+(objects[i].acceleration*dt)
        objects[i].position = objects[i].position+(objects[i].velocity*dt)

In [8]:
def update(bodies,dt=1/30):

# Initialize a force matrix, populate it with the forces of the corresponding index. So, at poition (i,j) is Fij
    forceMatrix=np.zeros((len(bodies),len(bodies),2))
    for j in range(len(bodies)):
        for i in range(len(bodies)):
            if i == j:
                forceMatrix[i][j]=0
            else:
                forceMatrix[i][j]=calculateGravity(bodies[i],bodies[j])
    
    # Calculate Total Forces
    totForceArray = np.zeros((len(bodies),2))
    acceleration = np.zeros((len(bodies),2))
    for i in range(len(bodies)):
        totForceArray[i]=forceMatrix[i].sum(axis=0)
        acceleration[i]=(1/bodies[i].mass)*totForceArray[i]
        bodies[i].acceleration = acceleration[i]
        bodies[i].velocity = bodies[i].velocity+(bodies[i].acceleration*dt)
        bodies[i].position = bodies[i].position+(bodies[i].velocity*dt)

In [57]:
def gravitySimulation(numParticles = 10, kind='random', numFrames=500, clean=False):
    """
    Description
    -----------
    Runs an entire gravity simulation by generating frames and saving them to the essential 'Images for Simulation' folder. 

    Parameters
    ----------
    numParticles : int
        Number of particles in the simulation. Default is 10 point mass bodies.
    kind : string
       Parameter that determines the kind of gravity simultion.
       Current support for:
           -'random': A random selection of point masses selected Gaussianly. Default Value
           -'polygonal': symmetrically distributed identical point masses. Must be hardcoded in
           -'grid': Point Masses are initialized at fixed intervals in a square grid pattern. However, because of this, the numParticles argument has to be a perfect square
    numframes : int
       Length of simulation. Default is 500.
    clean : Boolean True or False
        clean True gives

    Returns
    -------
    Nothing. But! This function does fill your 'Images for Simulation' folder that is essential for this function to work.
    Its these images that you run the ffmpeg command on
    """
    #The path below should be the path that YOU are saving every frame to. I didnt want to provide my personal one, so unfortunately this is the one thing you will have to do yourself
    dir = './Images for simulation'

    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))
    
    #Determine the style of simulation
    if kind=='grid'
    if kind=='polygonal':
        bodies = np.zeros(numParticles,dtype='object')
        for i in range(numParticles):
            bodies[i]=PointMassBody(1*10**12.5,np.array([0,0]),np.array([0,0]),np.array([0,0]))
            bodies[i].position=np.array([6*np.cos((2*np.pi/numParticles)*i),6*np.sin((2*np.pi/numParticles)*i)])
            bodies[i].velocity=np.array([6*(-1)*np.sin((2*np.pi/numParticles)*i),6*np.cos((2*np.pi/numParticles)*i)])
    else:
        bodies = generateParticles(numParticles)

    # Calculate the force between each every body and every other body
    #Start the main loop
    for i in range(numFrames):
        figure, axes = plt.subplots()
        update(bodies,dt=1/120)

        for j in range(len(bodies)):
            axes.scatter(bodies[j].position[0], bodies[j].position[1])

        if clean == True:
            plt.grid(None)
            plt.axis('off')
        else:
            pass
        axes.set_aspect(1)
        plt.xlim(-10,10)
        plt.ylim(-10,10)

        figure.savefig('./Images for simulation/graph'+str(i)+'.png', dpi=300)
        plt.close('all')

In [ ]:
# This cell runs gravitySimulation multiple times but with an increasing number of bodies up to 100 frames and stores the time it takes to run in an array to see just how bad it is to increase the body size. Can we see an n^2 curve?

# timeArray = np.array([])
# for i in range(148):
#     start = time.time()
#     sim.gravitySimulation(numParticles=(i+2))
#     end=time.time()
#     tot = end-start
#     timeArray = np.append(timeArray,tot)

In [36]:
timeArray = np.array([  7.80446482,   8.27741599,   8.8015027 ,   8.92677593,
         9.24045205,   9.90946746,  10.16627049,   9.81162167,
        11.43102169,  12.24573469,  11.25379276,  11.46036696,
        13.66306639,  12.29544067,  14.83429813,  13.07281256,
        13.54059386,  17.04186893,  14.97750044,  14.72370481,
        15.64790773,  19.12566614,  15.81220317,  17.18706083,
        16.56432557,  21.72000146,  17.5169332 ,  17.60667491,
        18.12724209,  18.48997831,  19.12640953,  25.22608876,
        19.74298263,  20.6413331 ,  20.60941815,  21.15631461,
        29.21979666,  22.72543764,  22.99953818,  23.58451724,
        24.61097813,  25.05568933,  24.75226045,  34.75182986,
        26.50055575,  26.06648183,  27.36986852,  29.9988203 ,
        29.09888911,  29.76648569,  29.86663651,  42.94832134,
        30.62651157,  30.99895477,  32.02199841,  32.44568896,
        32.70746946,  33.74574614,  33.61452842,  49.5013082 ,
        35.21585178,  35.54047203,  35.1264782 ,  36.95036077,
        36.47128773,  37.70188475,  37.68711424,  39.03866458,
        37.98228002,  59.10153651,  39.74072504,  41.61395144,
        40.78935933,  42.40180254,  43.4589591 ,  41.68976259,
        43.18684864,  43.42161131,  45.2848134 ,  43.50387216,
       105.19278741,  46.33233762,  47.2322526 ,  47.7939682 ,
        49.83375311,  48.85447216,  50.26118779,  54.37615824,
        52.16631675,  55.19746542,  52.13811874,  51.46402574,
        52.80079126, 190.49251437,  53.83967423,  55.59585714,
        56.24787474,  56.46714664])